<a href="https://colab.research.google.com/github/curtiscu/LYIT/blob/master/BulkVisualisations_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visualisations across bulk loaded data

# Setup env


In [1]:
# print all cell output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


## Google drive access

In [2]:
# mount google drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
# test, peek at data
! ls -al '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/'

# test, modules from local  'E:\Google Drive\LYIT\Dissertation\modules'
! ls -al '/content/drive/My Drive/LYIT/Dissertation/modules/'

total 35
-rw------- 1 root root 2589 Apr 27 12:01 10_soul-groove10_102_beat_4-4.mid
-rw------- 1 root root 4793 Apr 27 12:01 1_funk-groove1_138_beat_4-4.mid
-rw------- 1 root root 3243 Apr 27 12:01 2_funk-groove2_105_beat_4-4.mid
-rw------- 1 root root 4466 Apr 27 12:01 3_soul-groove3_86_beat_4-4.mid
-rw------- 1 root root 2551 Apr 27 12:01 4_soul-groove4_80_beat_4-4.mid
-rw------- 1 root root 3798 Apr 27 12:01 5_funk-groove5_84_beat_4-4.mid
-rw------- 1 root root 3760 Apr 27 12:01 6_hiphop-groove6_87_beat_4-4.mid
-rw------- 1 root root 1894 Apr 27 12:01 7_pop-groove7_138_beat_4-4.mid
-rw------- 1 root root 2437 Apr 27 12:01 8_rock-groove8_65_beat_4-4.mid
-rw------- 1 root root 3448 Apr 27 12:01 9_soul-groove9_105_beat_4-4.mid
total 43
-rw------- 1 root root 22877 Jul  2 21:06 data_prep.py
drwx------ 2 root root  4096 May 10 13:31 __pycache__
-rw------- 1 root root  7272 Jun 26 14:53 stats_n_features.py
-rw------- 1 root root  8416 Jun  2 21:24 visualizations.py


## Auto reload module

Now using library code I've created and saved to google drive which is automatically pushed to the cloud and made available to the colab env. The autoreload stuff below should help imports to 'reimport' to load changes to the library code.

It's not the quickest/ most reliable, so if in a hurry, brute force loading of changes by restarting the runtime.

In [4]:
# tool to auto reload modules.
%load_ext autoreload

# config to auto-reload all modules, handy to make 
# writing and testing modules much easier.
%autoreload 2

## Imports and accessing lib functions

In [5]:
# install required libs
!pip install mido

     |████████████████████████████████| 61kB 2.9MB/s 


In [6]:
# import my modules
import sys
sys.path.append('/content/drive/My Drive/LYIT/Dissertation/modules/')
import data_prep as dp

>> LOADING custom module, when: 2020-07-02 21:28:54.756802, module name: data_prep


In [7]:
# imports
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np


# object that provides colours for charts
from itertools import cycle



In [8]:
# testing auto reload of modules 
dp.test_function_call('bling')

Test function in data_prep.py called and worked! when: 2020-07-02 21:28:54.803776,  param:bling


In [9]:
import stats_n_features as sf

>> LOADING custom module, when: 2020-07-02 21:28:55.158152, module name: stats_n_features


In [10]:
sf.test_function_call('hello')

Test function in stats_n_features called and worked! when: 2020-07-02 21:28:55.184843,  param:hello


## Pandas display options

In [11]:
def set_pandas_display_options() -> None:
    # Ref: https://stackoverflow.com/a/52432757/
    display = pd.options.display

    display.max_columns = 1000
    display.max_rows = 2000
    display.max_colwidth = 1000
    display.width = None
    # display.precision = 2  # set as needed

set_pandas_display_options()
#pd.reset_option('all')


# Bulk Load Data

### Setup

In [12]:
metafile = '/content/drive/My Drive/groove-v1.0.0-midionly/groove/info.csv'

meta_df = pd.read_csv(metafile, dtype = {"drummer": "string", 
                                         "session" : "string", 
                                         "id": "string", 
                                         "style": "string", 
                                         "beat_type": "string", 
                                         "time_signature" : "string",
                                         "midi_filename" : "string",
                                         "audio_filename" : "string",
                                         "split" : "string"})

# show count of test/ train/ validation split specified in file...
meta_df.groupby(['split', 'drummer']).size()


split       drummer  
test        drummer1      48
            drummer3       8
            drummer5      13
            drummer7      40
            drummer8      17
            drummer9       3
train       drummer1     386
            drummer10     10
            drummer2      17
            drummer3      83
            drummer4       7
            drummer5      36
            drummer6      11
            drummer7     263
            drummer8      58
            drummer9      26
validation  drummer1      60
            drummer2       1
            drummer3       7
            drummer4       2
            drummer5       5
            drummer6       4
            drummer7      39
            drummer8       5
            drummer9       1
dtype: int64

In [13]:

# pull out the data for just 'eval_session' ..
eval_df = meta_df[meta_df['session'].str.contains('eval_session')].copy()
eval_df.info()

eval_df.drop(columns=['audio_filename', 'duration', 'beat_type'], inplace=True)

eval_df.groupby('drummer', as_index=False).count()
print('list of unique drummer IDs: {}'.format(eval_df['drummer'].unique()))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 0 to 1033
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   drummer         40 non-null     string 
 1   session         40 non-null     string 
 2   id              40 non-null     string 
 3   style           40 non-null     string 
 4   bpm             40 non-null     int64  
 5   beat_type       40 non-null     string 
 6   time_signature  40 non-null     string 
 7   midi_filename   40 non-null     string 
 8   audio_filename  40 non-null     string 
 9   duration        40 non-null     float64
 10  split           40 non-null     string 
dtypes: float64(1), int64(1), string(9)
memory usage: 3.8 KB


,drummer,session,id,style,bpm,time_signature,midi_filename,split
0,drummer1,10,10,10,10,10,10,10
1,drummer5,10,10,10,10,10,10,10
2,drummer7,10,10,10,10,10,10,10
3,drummer8,10,10,10,10,10,10,10


list of unique drummer IDs: <StringArray>
['drummer1', 'drummer5', 'drummer7', 'drummer8']
Length: 4, dtype: string


In [14]:
file_prefix = '/content/drive/My Drive/groove-v1.0.0-midionly/groove/'

eval_df['long_midi_filename'] = eval_df['midi_filename'].apply(lambda x: "{}{}".format(file_prefix, x))


In [15]:
# display(eval_df)

## Do the loading bit ...

In [16]:
all_drummer_data = {}

# iterate over rows with iterrows()
for index, row in eval_df.iterrows():

  # access data using column names
  next_drummer = row['drummer']
  next_file = row['long_midi_filename']

  # loads DataFrame (file_df), MIDI_File_Wrapper (f), and
  # the associated MidiTimingTools (mtt) objects..
  file_df, f, mtt = dp.load_file(next_file)

  err_bucks = sf.error_buckets(file_df) # parse for problem beats
  stats_dict = sf.gather_stats(file_df) # parse to gather stats

  # add tuple of data elements to dict with filename as key
  all_drummer_data[next_file] = (next_drummer, file_df, f, mtt, err_bucks, stats_dict)


FILE name: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/1_funk-groove1_138_beat_4-4.mid
    loaded file: <midi file '/content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/1_funk-groove1_138_beat_4-4.mid' type 0, 1 tracks, 1300 messages>
    time sig: <meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
    tempo: <meta message set_tempo tempo=434783 time=0>
    track count: 1, tracks: [<midi track 'MIDI' 1300 messages>]
    MIDI file type: 0
    > processing track: <midi track 'MIDI' 1300 messages>
    __notes pre-filter: [36. 37. 38. 40. 43. 44. 51. 52. 53. 55.]
    __applying filter: [44]
    __notes post filter: [36. 37. 38. 40. 43. 51. 52. 53. 55.]
    note_on span - first tick: 3 , last tick: 30658 
    good instruments: 5, {36.0: 'Bass Drum 1 (36)', 38.0: 'Acoustic Snare (38)', 43.0: 'High Floor Tom (43)', 49.0: 'Crash Cymbal 1 (49)', 51.0: 'Ride Cymbal 1 (51)'}
    ba

In [17]:
# take a peek at some loaded data
for next_file in all_drummer_data.keys():
  next_drummer, file_df, f, mtt, err_bucks, stats_dict = all_drummer_data[next_file]
  print('drummer: {}, file: {}'.format(next_drummer, next_file))
  details = err_bucks
  if details.size == 0:
    print('  ..nothing to see here')
  else:
    display(details)



drummer: drummer1, file: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/1_funk-groove1_138_beat_4-4.mid


track_msg_num  total_ticks  velocity  \
bar_number bar_beat_number note                                        
1          9               51            0:43          940        46   
                           51            0:46          971         0   
2          16              38           0:155         3710        37   
                           38           0:158         3747        41   
3          1               38           0:161         3796        33   
                           38           0:168         3846        23   
4          10              38           0:289         6813        37   
                           38           0:290         6867        20   
5          16              38           0:401         9468        37   
                           38           0:404         9506        33   
                           38           0:406         9532        30   
6          16              38           0:487        11397        36   
                           38           0:491        11441        32   
9          16              38           0:730        17147        38   
                           38           0:733        17182        38   
10         16              38           0:814        19056        43   
                           38           0:817        19092        46   
                           38           0:820        19116        40   
13         16              38          0:1056        24797        33   
                           38          0:1059        24845        32   
                           38          0:1062        24876        32   
14         16              38          0:1140        26733        40   
                           38          0:1143        26778        41   
                           38          0:1145        26805        34   

                                 beat_offset beat_center  bar_number  \
bar_number bar_beat_number note                                        
1          9               51            -20         960           1   
                           51             11         960           1   
2          16              38            -10        3720           2   
                           38             27        3720           2   
3          1               38            -44        3840           3   
                           38              6        3840           3   
4          10              38            -27        6840           4   
                           38             27        6840           4   
5          16              38            -12        9480           5   
                           38             26        9480           5   
                           38             52        9480           5   
6          16              38             -3       11400           6   
                           38             41       11400           6   
9          16              38            -13       17160           9   
                           38             22       17160           9   
10         16              38            -24       19080          10   
                           38             12       19080          10   
                           38             36       19080          10   
13         16              38            -43       24840          13   
                           38              5       24840          13   
                           38             36       24840          13   
14         16              38            -27       26760          14   
                           38             18       26760          14   
                           38             45       26760          14   

                                 bar_beat_number  
bar_number bar_beat_number note                   
1          9               51                  9  
                           51                  9  
2          16              38                 16  
                           38  

drummer: drummer1, file: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/10_soul-groove10_102_beat_4-4.mid


track_msg_num  total_ticks  velocity  \
bar_number bar_beat_number note                                        
1          14              38            0:35         1509        28   
                           38            0:38         1555        10   
3          6               38            0:93         4418        10   
                           38            0:94         4448         7   
4          13              38           0:148         7167       127   
                           38           0:151         7231        33   
11         13              38           0:414        20622       127   
                           38           0:417        20685        37   

                                 beat_offset beat_center  bar_number  \
bar_number bar_beat_number note                                        
1          14              38            -51        1560           1   
                           38             -5        1560           1   
3          6               38            -22        4440           3   
                           38              8        4440           3   
4          13              38            -33        7200           4   
                           38             31        7200           4   
11         13              38            -18       20640          11   
                           38             45       20640          11   

                                 bar_beat_number  
bar_number bar_beat_number note                   
1          14              38                 14  
                           38                 14  
3          6               38                  6  
                           38                  6  
4          13              38                 13  
                           38                 13  
11         13              38                 13  
                           38                 13

drummer: drummer1, file: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer1/eval_session/2_funk-groove2_105_beat_4-4.mid


...

drummer: drummer8, file: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer8/eval_session/10_soul-groove10_102_beat_4-4.mid


track_msg_num  total_ticks  velocity  \
bar_number bar_beat_number note                                        
1          10              38            0:29         1096        35   
                           38            0:30         1126        38   
2          9               38            0:69         2859        22   
                           38            0:72         2889        18   
3          11              38           0:116         4986        36   
                           38           0:118         5009        33   
5          11              38           0:200         8828        37   
                           38           0:202         8852        35   
                           38           0:204         8874        23   
7          2               36           0:265        11595         7   
                           36           0:269        11660        60   
           11              38           0:286        12663        25   
                           38           0:288        12686        20   
8          9               38           0:325        14371        21   
                           38           0:327        14401        15   
9          11              38           0:374        16507        28   
                           38           0:376        16531        24   
10         9               38           0:411        18184        30   
                           38           0:413        18208        29   
                           38           0:415        18225        23   
11         10              38           0:459        20296        27   
                           38           0:460        20330        17   
12         8               38           0:494        21987        19   
                           38           0:495        22019        20   
14         1               36           0:558        24936         8   
                           36           0:559        24968        88   
15         10              38           0:623        27988        23   
                           38           0:624        28019        22   
           11              38           0:626        28039        26   
                           38           0:628        28059        28   

                                 beat_offset beat_center  bar_number  \
bar_number bar_beat_number note                                        
1          10              38             16        1080           1   
                           38             46        1080           1   
2          9               38            -21        2880           2   
                           38              9        2880           2   
3          11              38            -54        5040           3   
                           38            -31        5040           3   
5          11              38            -52        8880           5   
                           38            -28        8880           5   
                           38             -6        8880           5   
7          2               36            -45       11640           7   
                           36             20       11640           7   
           11              38            -57       12720           7   
                           38            -34       12720           7   
8          9               38            -29       14400           8   
                           38              1       14400           8   
9          11              38            -53       16560           9   
                           38            -29       16560           9   
10         9               38            -56       18240          10   
                           38            -32       18240          10   
                           38            -15       18240          10   
11         10              38             16       20280          11   
                           38             50       20280          11   

drummer: drummer8, file: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer8/eval_session/2_funk-groove2_105_beat_4-4.mid
  ..nothing to see here
drummer: drummer8, file: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer8/eval_session/3_soul-groove3_86_beat_4-4.mid
  ..nothing to see here
drummer: drummer8, file: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer8/eval_session/4_soul-groove4_80_beat_4-4.mid
  ..nothing to see here
drummer: drummer8, file: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer8/eval_session/5_funk-groove5_84_beat_4-4.mid
  ..nothing to see here
drummer: drummer8, file: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer8/eval_session/6_hiphop-groove6_87_beat_4-4.mid


track_msg_num  total_ticks  velocity  \
bar_number bar_beat_number note                                        
9          8               36           0:418        16193        64   
                           36           0:421        16234        18   

                                 beat_offset beat_center  bar_number  \
bar_number bar_beat_number note                                        
9          8               36             -7       16200           9   
                           36             34       16200           9   

                                 bar_beat_number  
bar_number bar_beat_number note                   
9          8               36                  8  
                           36                  8

drummer: drummer8, file: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer8/eval_session/7_pop-groove7_138_beat_4-4.mid
  ..nothing to see here
drummer: drummer8, file: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer8/eval_session/8_rock-groove8_65_beat_4-4.mid
  ..nothing to see here
drummer: drummer8, file: /content/drive/My Drive/groove-v1.0.0-midionly/groove/drummer8/eval_session/9_soul-groove9_105_beat_4-4.mid
  ..nothing to see here
